### Evaluating CPM with CEBaB
Our Causal Proxy Model (CPM) is for providing concept-based explanation for a blackbox model. We use newly developed CEBaB benchmark for comparing CPM with other concept-based explanation methods. This notebook evaluates CPM with CEBaB benchmark under different settings.

More importantly, we introduce new baselines for CPM as well. Formally, we evaluate the blackbox model with interchange intervention evaluation (which will be introduced in details below).

In this notebook, we can evaluate the following models:
- CPM: `BERT-base-uncased`
- CPM: `RoBERTa-base`
- CPM: `GPT2`
- CPM: `LSTM+GloVe`
- CPM: `Control`

and we can evaluate with the following conditions:
- 2-class
- 3-class
- 5-class

and this script also support evaluates experiments with different control conditions:
- `control=True`
- `control=False`
- `control="pretrain"`
- `control="random"`
- `control="finetune"`
- `control="fewshots"` (supports BERT only)
- `control="fewshots-augment"` (supports BERT only)
- `control="layer"` (supports BERT only)

#### Imports and Libs

In [37]:
from libs import *
from modelings.modelings_bert import *
from modelings.modelings_roberta import *
from modelings.modelings_gpt2 import *
from modelings.modelings_lstm import *
"""
For evaluate, we use a single random seed, as
the models are trained with 5 different seeds
already.
"""
_ = random.seed(123)
_ = np.random.seed(123)
_ = torch.manual_seed(123)

#### Main evaluate script

In [38]:
"""
The following blocks will run CEBaB benchmark in
all the combinations of the following conditions.
"""
grid = {
    "control": ["true-counterfactuals"],
    "seed": [42],
    "h_dim": [192],
    "interchange_layer" : [12],
    "class_num": [5],
    "k" : [19684], 
    "beta" : [1.0],
    "gemma" : [3.0],
    "model_arch" : ["bert-base-uncased"],
    "lr" : ["8e-5"]
    "counterfactual_type" : ["true"]
}

keys, values = zip(*grid.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

device = 'cuda:0'
batch_size = 32

In [35]:
results = {}
for i in range(len(permutations_dicts)):
    
    seed=permutations_dicts[i]["seed"]
    class_num=permutations_dicts[i]["class_num"]
    beta=permutations_dicts[i]["beta"]
    gemma=permutations_dicts[i]["gemma"]
    h_dim=permutations_dicts[i]["h_dim"]
    dataset_type = f'{class_num}-way'
    control=permutations_dicts[i]["control"]
    model_arch=permutations_dicts[i]["model_arch"]
    k=permutations_dicts[i]["k"]
    interchange_layer=permutations_dicts[i]["interchange_layer"]
    lr=permutations_dicts[i]["lr"]
    counterfactual_type=permutations_dicts[i]["counterfactual_type"]
    
    if model_arch == "bert-base-uncased":
        model_path = "BERT"
        model_module = BERTForCEBaB
        explainer_module = CausalProxyModelForBERT
    elif model_arch == "roberta-base":
        model_path = "RoBERTa" 
        model_module = RoBERTaForCEBaB
        explainer_module = CausalProxyModelForRoBERTa
    elif model_arch == "gpt2":
        model_path = "gpt2"
        model_module = GPT2ForCEBaB
        explainer_module = CausalProxyModelForGPT2
    elif model_arch == "lstm":
        model_path = "lstm"
        model_module = LSTMForCEBaB
        explainer_module = CausalProxyModelForLSTM
    model_path += f"-{control}"
    grid_conditions=(
        ("control", control),
        ("seed", seed),
        ("h_dim", h_dim),
        ("interchange_layer", interchange_layer),
        ("class_num", class_num),
        ("k", k),
        ("beta", beta),
        ("gemma", gemma),
        ("model_arch", model_arch),
        ("lr", lr)
        ("counterfactual_type", counterfactual_type)
    )
    print("Running for this setting: ", grid_conditions)

    blackbox_model_path = f'CEBaB/{model_arch}.CEBaB.sa.'\
                          f'{class_num}-class.exclusive.seed_{seed}'
    cpm_model_path = f'../proxy_training_results/{model_path}/'\
                     f'cebab.alpha.1.0.beta.{beta}.gemma.{gemma}.'\
                     f'lr.{lr}.dim.{h_dim}.hightype.{model_arch}.'\
                     f'CEBaB.cls.dropout.0.1.enc.dropout.0.1.counter.type.'\
                     f'{counterfactual_type}.k.{k}.int.layer.{interchange_layer}.'\
                     f'seed_{seed}/'

    # load data from HF
    cebab = datasets.load_dataset(
        'CEBaB/CEBaB', use_auth_token=True,
        cache_dir="../train_cache/"
    )
    train, dev, test = preprocess_hf_dataset(
        cebab, one_example_per_world=True, 
        verbose=1, dataset_type=dataset_type
    )

    tf_model = model_module(
        blackbox_model_path, 
        device=device, 
        batch_size=batch_size
    )
    explanator = explainer_module(
        blackbox_model_path,
        cpm_model_path, 
        device=device, 
        batch_size=batch_size,
        intervention_h_dim=h_dim,
    )

    train_dataset = train.copy()
    dev_dataset = test.copy()

    result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
    CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
    ACaCE_per_aspect, performance_report = cebab_pipeline(
        tf_model, explanator, 
        train_dataset, dev_dataset, 
        dataset_type=dataset_type,
        correction_epsilon=correction_epsilon,
    )
    
    results[grid_conditions] = (
        result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
        CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
        ACaCE_per_aspect, performance_report
    )

Running for this setting:  (('seed', 42), ('class_num', 5), ('beta', 1.0), ('gemma', 3.0), ('h_dim', 192), ('dataset_type', '5-way'), ('correction_epsilon', None), ('cls_dropout', 0.1), ('enc_dropout', 0.1), ('control', 'dev'), ('model_arch', 'bert-base-uncased'), ('true_cfc', 1755), ('interchange_layer', 12))


Using custom data configuration CEBaB--CEBaB-0e2f7ed67c9d7e55
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-0e2f7ed67c9d7e55/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/4 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train dataset.
intervention_h_dim=192


100%|██████████| 124/124 [00:24<00:00,  5.05it/s]


#### Show your results

In [36]:
important_keys = [
    "seed", "h_dim", "class_num", 
    "control", "beta", "gemma", 
    "cls_dropout", "enc_dropout", 
    "model_arch", "true_cfc", 
    "interchange_layer"
]
values = []
for k, v in results.items():
    _values = []
    for ik in important_keys:
        _values.append(dict(k)[ik])
    _values.append(v[2]["ICaCE-L2"].iloc[0])
    _values.append(v[2]["ICaCE-cosine"].iloc[0])
    _values.append(v[2]["ICaCE-normdiff"].iloc[0])
    _values.append(v[-1].iloc[0][0])
    values.append(_values)
important_keys.extend(["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"])
df = pd.DataFrame(values, columns=important_keys)
df.sort_values(by=['class_num'], ascending=True)

,seed,h_dim,class_num,control,beta,gemma,cls_dropout,enc_dropout,model_arch,true_cfc,interchange_layer,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,42,192,5,dev,1.0,3.0,0.1,0.1,bert-base-uncased,1755,12,0.7219,0.4734,0.5105,0.681608


#### Save your results somewhere and load again to tabularize your results altogether

In [28]:
output_name = input("Plase give an output file name: ")

output_directory = f'../proxy_training_results/{model_path}/'
output_filename = os.path.join(output_directory, f'{output_name}.pkl')
print("Writing to file: ", output_filename)
with open(output_filename, 'wb') as f:
    pickle.dump(results, f)

Plase give an output file name: results
Writing to file:  ../proxy_training_results/BERT-fewshots-hdim-results/results.pkl
